# MOWNIT LABORATORIUM 6 - SVD zastosowania

Opis nie będzie bardzo obfity ponieważ mam nadzieje że więcej będzie widoczne na prezentacji

krok 1. Crawlowanie sieci + tworzenie strony: plik lab6_websiteHandler.py

Kod ten crawluje linki (getURLList), scrapuje potem te linki (urlFromWebiste) i tworzy odpowiednie słowniki słów (TextToBow, urlToBowList). Ztokenizowanie słów do wielkiego słownika klucz:wartość -> linkDoStrony:słownikSłów (createBOWs). Podczas tego tworze także Inverse document frequency i unie wszystkich słów (CreateBOWs i calculateIDFs). Każda operacja jest na słownikach i zapisywana do odpowiedniego pliku JSON (save_dict_to_file, load_dict_from_file) (ponieważ operacje przypisu i odczytu na hashsetach sa szybsze (O(1)) niż na tablicach (O(N))).

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from queue import PriorityQueue
import re
import multiprocessing
import json
import math


def getStopWords(path):
    List = []
    with open(path, "r") as file:
        for line in file:
            List.append(line.strip())
    return List


# globals
stopWords = getStopWords("Databases/stopwords.txt")


def stopWordsRemoval(x):
    global stopWords
    return not x in stopWords


def TextToBOW(text: str) -> dict:
    def remove_unicode(text):
        return re.sub(r'[^\x00-\x7F]', ' ', text)

    clean_text = remove_unicode(text.lower())
    words = re.split(
        r'[\/;\'\n:().,\s\[\]\-"`?@! ]+', clean_text)
    FilteredText = list(filter(stopWordsRemoval, words))
    BOW = {}
    for word in FilteredText:
        BOW[word] = 1 + BOW.get(word, 0)

    return BOW


def UrlToWorldList(url):
    try:
        page = urlopen(url)
    except:
        return
    html_bytes = page.read()
    try:
        html = html_bytes.decode("utf-8")
    except:
        return
    soup = BeautifulSoup(html, "lxml")
    text = soup.getText()

    return TextToBOW(text)


def save_dict_to_file(dictionary, filename):
    with open(filename, 'w') as file:
        json.dump(dictionary, file)


def load_dict_from_file(filename) -> dict:
    with open(filename, 'r') as file:
        return json.load(file)


def filter_none_values(dictionary: dict):
    return {k: v for k, v in dictionary.items() if v is not None}


def urlFromWebsite(InitialURL: str, amount: int) -> dict:
    print("Finding in: ", InitialURL)
    URLs = {}
    Q = PriorityQueue()
    Q.put((1, InitialURL))
    URLs[InitialURL] = True
    while len(URLs) <= amount and (not Q.empty()):

        print(len(URLs))
        deph, url = Q.get()
        if deph > 3:
            continue
        try:
            page = urlopen(url)
        except:
            # print(url)
            continue

        html_bytes = page.read()
        try:
            html = html_bytes.decode("utf-8")
        except:
            continue
        soup = BeautifulSoup(html, "lxml")
        for link in soup.findAll("a"):
            parsedLink = link.get('href')
            if parsedLink is None:
                continue

            # links sometimes start with for example "/news" or "//bbc.com/news" instead of "https://bbc.com/news"
            # thats why i want to look into both cases, this way it is the fastest

            if InitialURL in parsedLink:
                pass
            elif parsedLink.startswith("//") and parsedLink != "//":
                parsedLink = "https:" + parsedLink
            elif parsedLink.startswith("/") and parsedLink != "/":
                parsedLink = InitialURL + parsedLink
            else:
                continue

            if not URLs.get(parsedLink, False):
                Q.put((deph+1, parsedLink))
                URLs[parsedLink] = True

    print("Parsing: ", InitialURL)
    print("Warning it might take a lot of time (20mins +)")

    for url in list(URLs.keys()):
        URLs[url] = UrlToWorldList(url)

    URLs = filter_none_values(URLs)

    print("Finished looking for URLs in: ",
          InitialURL, ". Found ", len(URLs), " URLs")
    return URLs


def process_url(URL, amount, result_dict):
    result_dict.update(urlFromWebsite(URL, amount))


def getUrlList():
    InitialURLs = [("https://www.bbc.com", 1000),
                   ("https://www.nbcnews.com", 1000),
                   ("https://edition.cnn.com", 1000),
                   ("https://www.foxnews.com", 2000),
                   ("https://people.com", 1500),
                   ("https://www.abc.net.au", 500),
                   ("https://www.news18.com", 2000),
                   ("https://eu.usatoday.com", 700),
                   ("https://news.sky.com", 1000)]

    manager = multiprocessing.Manager()
    URLs = manager.dict()
    jobs = []

    for URL, amount in InitialURLs:
        p = multiprocessing.Process(
            target=process_url, args=(URL, amount, URLs))
        jobs.append(p)
        p.start()

    for p in jobs:
        p.join()

    return dict(URLs)


def createBOWs():
    global globalBow, wordIndex
    print("================GettingBOWs==============")

    # Get URLs using multiprocessing
    URLs = getUrlList()
    URLIndexes = {}
    URLIndex = 0
    globalBOW = {}
    for url in list(URLs.keys()):
        URLIndexes[url] = URLIndex
        URLIndex += 1
        globalBOW.update(URLs[url])

    BOWIndex = 0
    for word in list(globalBOW.keys()):
        globalBOW[word] = BOWIndex
        BOWIndex += 1

    print(globalBOW)
    print(BOWIndex)
    save_dict_to_file(URLs, "Databases/URLs.json")
    save_dict_to_file(globalBOW, "Databases/globalBow.json")
    print("=======================DONE=======================")


def calculateIDFs() -> dict:
    print("=====Calculationg Inverse document frequencies====")
    Occurances = {}
    URLs = load_dict_from_file("Databases/URLs.json")
    for _, urlDict in URLs.items():
        for word in list(urlDict.keys()):
            Occurances[word] = 1 + Occurances.get(word, 0)

    N = len(URLs)
    IDF = {}

    for key in Occurances:
        IDF[key] = math.log10(N/Occurances[key])

    save_dict_to_file(IDF, "Databases/IDFs.json")
    print("=======================DONE=======================")

    return IDF


def IndexUrls():
    print("=========INDEXING URLS=========")
    URLs = load_dict_from_file("Databases/URLs.json")
    URLIndexes = {}
    IndexedURLs = {}
    currIndex = 0
    for URL, _ in URLs.items():
        URLIndexes[URL] = currIndex
        IndexedURLs[currIndex] = URL
        currIndex += 1

    save_dict_to_file(IndexedURLs, "Databases/IndexedURLs.json")
    save_dict_to_file(URLIndexes, "Databases/URLIndexes.json")
    print("=======================DONE=======================")

## Krok 2. Operacje na wyszukiwaniach

plik - lab6.py

Ta część kodu odpowiada za wszystkie operacje na macierzach - używam główne biblioteki scipy.sparse csr_matrix, csc_matrix w zależności od potrzeby (i lil_matrix przy tworzeniu). Dodatkowo także liczę $U_k\Sigma_k$ i $V_k^T$ używając biblioteki sklearn.decomposition, ponieważ jak podaje dokumentacja: 

*This transformer performs linear dimensionality reduction by means of truncated singular value decomposition (SVD). Contrary to PCA, this estimator does not center the data before computing the singular value decomposition. This means it can work with sparse matrices efficiently.*

Następnie mamy funkcje
```py
def PrzeglądarkaMain(TermByDocumentMatrix, InputString: str, URLsLen, SVD=False, OutputSize=10) -> list:
```
która jest sercem mojej "wyszukiwarki". Przyjmuje ona na wejściu string, następnie zamienia ten string na BOW i go normalizuje, poczym wykonuje 2 dostepne kalkulacje wektora podobieństwa:

bez SVD:
$$|q^TA| = [|cos\theta_1|,|cos\theta_2|,...,|cos\theta_n|]$$

oraz z SVD:
$$|q^TU_k\Sigma_kV^T_k| = [|cos\theta_1|,|cos\theta_2|,...,|cos\theta_n|]$$

Następnie odpowiedniemu miejscu w wektorze przypisujemy jego strone i wybieramy OutputSize największych


*dodatkowo zostało zakomentowane rozwiązanie podpunktu 6. które jest zdecydowanie wolniejsze i odpowiada temu co się wykonuje w zadaniu 7, oraz w zad 8 nowa miara podobieństwa została zmieniona w odpowiednio ten sam sposób*

In [ ]:
from scipy import sparse
from scipy.sparse import linalg
from sklearn.decomposition import TruncatedSVD

import numpy as np

from lab6_websiteHandler import *


def BOWtoVector(BOW) -> sparse.csr_matrix:
    globalBow = load_dict_from_file("Databases/globalBow.json")
    IDFs = load_dict_from_file("Databases/IDFs.json")
    SparseVector = sparse.lil_matrix((len(globalBow), 1), dtype=np.float32)
    for word, value in BOW.items():
        if word not in globalBow:
            continue
        SparseVector[globalBow[word], 0] = value*IDFs[word]
    return sparse.csr_matrix(SparseVector.tocsr())


def CreateTermByDocumentMatrix() -> sparse.csr_matrix:
    print("===============CREATING TBD MATRIX================")
    URLs = load_dict_from_file("Databases/URLs.json")
    globalBow = load_dict_from_file("Databases/globalBow.json")
    URLIndexes = load_dict_from_file("Databases/URLIndexes.json")
    IDFs = load_dict_from_file("Databases/IDFs.json")
    TermByDocumentMatrix = sparse.lil_matrix(
        (len(globalBow), len(URLs)), dtype=np.float32)
    for url, urlDict in URLs.items():
        for word, value in urlDict.items():
            TermByDocumentMatrix[globalBow[word],
                                 URLIndexes[url]] = value*IDFs[word]

    TermByDocumentMatrix_CSC = TermByDocumentMatrix.tocsc()
    column_norms = linalg.norm(TermByDocumentMatrix_CSC, axis=0)
    normalisedTermByDocumentMatrix = TermByDocumentMatrix_CSC/column_norms
    print("=======================DONE=======================")
    return normalisedTermByDocumentMatrix


def initialize_svd(search_matrix, k):

    print("SVD calculation started.")
    svd = TruncatedSVD(n_components=k)
    svd.fit(search_matrix)

    us_matrix = svd.transform(search_matrix)
    v_t_matrix = np.array(svd.components_)
    print("SVD calculation finished.")

    with open("Databases/us.npz", 'wb') as file:
        np.save(file, us_matrix)
    with open("Databases/v_t.npz", 'wb') as file:
        np.save(file, v_t_matrix)
    print("SVD matrices saved succesfully.")


def calculateCosine(Query: sparse.csr_matrix, TBDMatrix: sparse.csr_matrix, Ej: sparse.csr_matrix):
    return (Query.transpose() @ (TBDMatrix @ Ej.transpose()))/(linalg.norm(Query) * linalg.norm(TBDMatrix @ Ej.transpose()))


def PrzeglądarkaMain(TermByDocumentMatrix, InputString: str, URLsLen, SVD=False, OutputSize=10) -> list:
    # this slow and bad
    # def handleQuery(Query: sparse.csr_matrix):
    #     nonlocal TermByDocumentMatrix, URLs, IndexedURLs
    #     QueryResult = {}
    #     for j in range(len(URLs)):
    #         QueryResult[IndexedURLs[str(j)]] = calculateCosine(
    #             Query, TermByDocumentMatrix, TermByDocumentMatrix[j:j+1, :])
    #     return QueryResult

    def handleQuery2(Query: sparse.csr_matrix):
        nonlocal TermByDocumentMatrix, IndexedURLs
        normalisedQuery = Query/linalg.norm(Query)
        Cosines = normalisedQuery.transpose() @ TermByDocumentMatrix
        QueryResult = {}
        for j in range(len(Cosines)):
            QueryResult[IndexedURLs[str(j)]] = Cosines[0, j]
        return QueryResult

    def handleQuerySVD(Query: sparse.csr_matrix):
        nonlocal TermByDocumentMatrix, IndexedURLs
        us_matrix = np.load("Databases/us.npz")
        v_t_matrix = np.load("Databases/v_t.npz")
        Query /= linalg.norm(Query)
        transposedQuery = np.transpose(Query.toarray())
        print(transposedQuery.shape)
        print(us_matrix.shape)
        print(v_t_matrix.shape)
        Cosines = (transposedQuery @ us_matrix) @ v_t_matrix
        QueryResult = {}
        for j in range(Cosines):
            QueryResult[IndexedURLs[str(j)]] = Cosines[0, j]
        return QueryResult

    IndexedURLs = load_dict_from_file("Databases/IndexedURLs.json")
    if InputString == "":
        return []

    Query = BOWtoVector(TextToBOW(InputString))

    if SVD == True:
        QueryResult = handleQuerySVD(Query)
    else:
        QueryResult = handleQuery2(Query)
    sortedQueryResult = sorted(
        QueryResult.items(), key=lambda x: x[1], reverse=True)
    result_list = sortedQueryResult[:OutputSize]

    return result_list

## Krok 3. Frontend

bardzo prosty frontend napisany przy użyciu Flaska zawierający 5 plików:
1. templates/index.html
```html
<!DOCTYPE html>
<html lang="en">
    <head>
        <link
            rel="stylesheet"
            href="{{ url_for('static', filename='styleIndex.css') }}"
        />
        <meta charset="UTF-8" />
        <meta name="viewport" content="width=device-width, initial-scale=1.0" />
        <title>Wyszukiwarka studenta</title>
    </head>
    <body>
        <div class="mainDiv">
            <h1><a id="noAnchor" href="/">Stuudentle</a></h1>
            <form action="/search" method="get">
                <div class="inputDiv">
                    <input type="text" name="q" />
                    <button type="submit">Submit</button>
                    <p>Specify how much documents you want to see:</p>
                    <input type="text" name="outputSize" style="width: 10%" />
                </div>
            </form>
            <form action="/remakeDatabase" method="get">
                <button type="submit">Remake Database</button>
            </form>
        </div>
    </body>
</html>
```
2. search.html
```html
<!DOCTYPE html>
<html lang="en">
    <head>
        <link
            rel="stylesheet"
            href="{{ url_for('static', filename='styleSearch.css') }}"
        />
        <meta charset="UTF-8" />
        <meta name="viewport" content="width=device-width, initial-scale=1.0" />
        <title>Wyszukiwarka studenta</title>
    </head>
    <body>
        <div class="mainDiv">
            <h1><a id="noAnchor" href="/">Stuudentle</a></h1>
            <form action="/search" method="get">
                <div class="inputDiv">
                    <input type="text" name="q" value="{{ input_string }}" />
                    <button type="submit">Submit</button>
                    <p>Specify how much documents you want to see:</p>
                    <input type="text" name="outputSize" style="width: 10%" />
                </div>
            </form>
            <br /><br />
            {% if links %}
            <h2>Search results</h2>
            <h3>Response time: {{ response_time }}</h3>
            <br />
            <ul>
                {% for link in links %}
                <li>
                    <p>[{{ link.value }}]</p>
                    <a href="{{ link.url }}">{{ link.url }}</a>
                </li>
                {% endfor %}
            </ul>
            {% endif %}
        </div>
    </body>
</html>
```
Odpowiadające im pliki css:

3. styleIndex.css
```css
body {
    background-color: #202124; 
    font-family: Arial, sans-serif;
    margin: 0;
    padding: 0;
}

.container {
    max-width: 800px;
    margin: 0 auto;
    padding: 20px;
}

h1 {
    color: #ffffff; 
    font-size: 400%;
}

form {
    margin-bottom: 20px;
}

input[type="text"] {
    padding: 10px;
    width: 70%;
    background-color: #292929; 
    color: #ffffff; 
    border: 1px solid #555; 
}
/*...*/
```
4. styleSearch.css
```css
body {
    background-color: #202124;
    font-family: Arial, sans-serif;
    margin: 0;
    padding: 0;
}

.container {
    max-width: 800px;
    margin: 0 auto;
    padding: 20px;
}

h1 {
    color: #ffffff;
    font-size: 250%;
}

form {
    margin-bottom: 20px;
}

input[type="text"] {
    padding: 10px;
    width: 70%;
    background-color: #292929;
    color: #ffffff;
    border: 1px solid #555;
}

button {
    padding: 10px 20px;
    background-color: #1a73e8;
    color: #ffffff;
    border: none;
    cursor: pointer;
}
/*...*/

```

Oraz flaskapp.py

In [ ]:
from flask import Flask, request, render_template, redirect, url_for
from lab6 import CreateTermByDocumentMatrix, PrzeglądarkaMain, initialize_svd
from lab6_websiteHandler import createBOWs, calculateIDFs, IndexUrls, load_dict_from_file
import time

app = Flask(__name__)


#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# . .venv/bin/activate !!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Create the Term-By-Document Matrix (TBDM) and store it in the app configuration
app.config['TBDM'] = CreateTermByDocumentMatrix()


def process_links(resultList):
    links = []
    if resultList is None:
        return
    for link, sparseMatrixValue in resultList:
        link = {'url': link,
                'value': str(round(sparseMatrixValue*100, 2)) + '%'}
        links.append(link)
    return links


def is_integer_string(s):
    return s.isdigit()


@app.route('/', methods=['GET', 'POST'])
def index():
    input_string = ""
    if request.method == 'POST':
        input_string = request.form['input_string']
    return render_template('index.html')


@app.route('/search', methods=['GET', 'POST'])
def search():
    # Retrieve the TBDM from the app configuration or initialize it if not present
    TBDM = app.config.get('TBDM')
    result = None
    time_taken = None
    k = 10
    if request.method == 'GET':
        URLs = load_dict_from_file("Databases/URLs.json")
        query = request.args.get('q', '')
        k = request.args.get('outputSize', 10)
        if not is_integer_string(k):
            k = 10
        start = time.time()
        result = PrzeglądarkaMain(TBDM, query, len(URLs), OutputSize=int(k))
        end = time.time()
        time_taken = str(round(end-start, 4)) + "s"
    return render_template('search.html', links=process_links(result), input_string=query, response_time=time_taken)


@app.route('/remakeDatabase')
def remakeDatabase():
    TBDM = app.config.get('TBDM')
    createBOWs()
    calculateIDFs()
    IndexUrls()
    initialize_svd(TBDM, 100)
    return redirect(url_for('index'))


if __name__ == '__main__':
    app.run(debug=True)

## Wyniki:

Czas trwania: 
- Części przygotowania danych poniżej 20 minut,
- Części uruchomienia przeglądarki (tworzenie macierzy) około 10 sekund
- Części liczenia rankingu dokumentów niecała sekunda

Wyniki:
- Z SVD (najlepsze wyniki koło k = 100): najlepsze wyszukania - wyniki w granicach 20-30%
- Bez SVD: wyszukania średnio 10-15% lepsze

IDF: nieznacząco wydłuża czas wyszukiwania, ale zwiększa dokładośc o około 5% względem wersji bez